In [43]:
"""
code is taken and adopted from https://github.com/bplank/bleaching-text/
"""




import re
from sklearn.model_selection import train_test_split
from collections import Counter
from math import log
from emoji import UNICODE_EMOJI
from scipy import stats
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction import DictVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report, f1_score
from sklearn.model_selection import StratifiedKFold
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline , FeatureUnion

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer


emoticon_string = r"""
    (?:
      [<>]?
      [:;=8>]                    # eyes
      [\-o\*\']?                 # optional nose
      [\)\]\(\[dDpPxX/\:\}\{@\|\\] # mouth      
      |
      [\)\]\(\[dDpPxX/\:\}\{@\|\\] # mouth
      [\-o\*\']?                 # optional nose
      [:;=8<]                    # eyes
      [<>]?
      |
      <[/\\]?3                         # heart(added: has)
      |
      \(?\(?\#?                   #left cheeck
      [>\-\^\*\+o\~]              #left eye
      [\_\.\|oO\,]                #nose
      [<\-\^\*\+o\~]              #right eye
      [\#\;]?\)?\)?               #right cheek
    )"""

emoticon_re = re.compile(emoticon_string, re.VERBOSE | re.I | re.UNICODE)
special=set(['NEWLINE','URL','USER'])

#prep for frequency
def initFreq(text):
    frequency=Counter()
    for line in text:
#         print(line)
        frequency.update([e.lower() for e in line.split(' ') if e not in special])
    for token,freq in list(frequency.items()):
        frequency[token]=str(freq)
#         frequency[token]=str(int(log(freq,10)))
    for token in special:
        frequency[token.lower()]=token
    return frequency

def frequency(word, frequency):
    return(frequency[word.lower()])

def length(word):
    return '0' + str(len(word))

def lex(word):
    return word

def punctAgr(token):
    emojis = ''
    for char in token:
        if char in UNICODE_EMOJI:
            emojis += 'J'
    if emojis != '':
        return(emojis)
    elif (emoticon_re.search(token) != None):
        return('E')
    else:
        isWord = False
        for char in token:
            if char.isalpha() or char.isalnum():
                isWord = True
        if isWord:
            beg = ''
            for i in range(len(token)):
                if token[i].isalpha() or token[i].isalnum():
                    break
                else:
                    beg += 'P'
            end = ''
            for i in range(len(token)-1, 0, -1):
                if token[i].isalpha() or token[i].isalnum():
                    break
                else:
                    end += 'P'
            return(beg + 'W' + end)
        else:
            return('P' * len(token)) 

def punctCons(token):
    emojis = ''
    for char in token:
        if char in UNICODE_EMOJI:
            emojis += char
    if emojis != '':
        return(emojis)
    else:
        newWord = ''
        lastAlpha = False
        for char in token:
            if char.isalpha() or char.isalnum():
                if not lastAlpha:
                    newWord += 'W'
                lastAlpha = True
            else:
                newWord += char
                lastAlpha = False
        return newWord

def shape(token):
    packed=''
    for char in token:
        if char.isupper():
            packed+='u'
        elif char.islower():  
            packed+='l'
        elif char.isdigit():
            packed+='d'
        else:
            packed+='x'
    return re.sub(r'(.)\1{2,}',r'\1\1',packed)
        
def vowels(word):
    new = ''
    for char in word:
        if char.lower() in 'euioa':
            new += 'v'
        elif char.isalpha():
            new += 'c'
        else:
            new += 'o'
    return new

def bleachAll(text, freq):
    newText = ''
#     for method in ['frequency', 'length', 'punctAgr', 'punctCons', 'shape', 'vowels','lex']:
#     for method in [ 'length', 'punctAgr', 'punctCons', 'shape', 'vowels','lex']:    
#     for method in ['frequency', 'length', 'punctAgr', 'punctCons', 'shape', 'vowels']:
    for method in [ 'length', 'punctAgr', 'punctCons', 'shape', 'vowels']:
#     for method in ['lex']:
#         if method:# == 'frequency':
            newText += bleachText(text, method, freq)
    return newText

# to use frequency, you have to run it like this:
# freqs = bleaching.initFreq(text)
# bleaching.bleachText(text, 'frequency', freqs)
# the frequency is then based on the entire dataset

#if you do not need the frequency transformation:
# bleaching.bleachText(text, 'length', None)
def bleachText(text, method, freq):
    if method == 'all':
        return bleachAll(text, freq)
    newText = ''
    for line in text.split('\n'):
        for word in line.split(' '):
            if word not in special:
                if method== 'frequency':
                    if freq == None:
                        print('ERROR: frequency is used, but no counter is given')
                    newText += str(frequency(word, freq)) + ' '
                if method== 'all':
                    if freq == None:
                        print('ERROR: all is used, but no counter is given')
                    
                    newText += str(frequency(word, freq)) + ' '
                elif method== 'length':
                    newText += length(word) + ' '
                elif method== 'lex':
                    newText += lex(word) + ' '
                elif method== 'punctAgr':
                    newText += punctAgr(word) + ' '
                elif method== 'punctCons':
                    newText += punctCons(word) + ' '
                elif method== 'shape':
                    newText += shape(word) + ' ' 
                elif method== 'vowels':
                    newText += vowels(word) + ' '
                elif method not in ["all",'frequency', 'length', 'lex', 'punctAgr', 'punctCons', 'shape', 'vowels']:
                    print('ERROR method ' + method + '  does not exist')
            else:
                newText += word + ' '
        newText += ' '
    return newText[:-1]

import nltk
from sklearn.base import TransformerMixin
import numpy as np
import json

# PREFIX_WORD_NGRAM="W:"
PREFIX_WORD_NGRAM=" "
PREFIX_CHAR_NGRAM="C:"
TWEET_DELIMITER = " NEWLINE "



def get_size_tuple(ngram_str):
    """
    Convert n-gram string to tuple
    :param ngram_str:  "1-3" (lower and upper bound separated by hyphen)
    :return: tuple
    >>> get_size_tuple("3-5")
    (3, 5)
    >>> get_size_tuple("1")
    (1, 1)
    """
    if "-" in ngram_str:
        lower, upper = ngram_str.split("-")
        lower = int(lower)
        upper = int(upper)
    else:
        lower = int(ngram_str)
        upper = lower
    return (lower, upper)

class Featurizer(TransformerMixin):
    """Our own featurizer: extract features from each document for DictVectorizer"""

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        """
        for all tweets of a user
        """
        out= [self._ngrams(tweets) for tweets in X]
        return out

    def __init__(self,word_ngrams="1",char_ngrams="0",binary=True,rm_user_url=False):
        """
        binary: whether to use 1/0 values or counts
        lowercase: convert text to lowercase
        remove_stopwords: True/False
        """
        self.data = [] # will hold data (list of dictionaries, one for every instance)
        self.binary=binary
        self.word_ngram_size = get_size_tuple(word_ngrams)
        self.char_ngram_size = get_size_tuple(char_ngrams)

        self.rm_user_url=rm_user_url


    def _ngrams(self,tweets):
        """
        extracts word or char n-grams
        range defines lower and upper n-gram size
        >>> f=Featurizer(word_ngrams="1-3")
        >>> d = f._ngrams("this is a test")
        >>> len(d)
        9
        >>> f=Featurizer(word_ngrams="0", char_ngrams="2-4")
        >>> d2 = f._ngrams("this")
        >>> len(d2)
        6
        """

        d={} # new dictionary that holds features for current instance

        tweets = tweets.split(TWEET_DELIMITER)

        lower, upper = self.word_ngram_size
        if lower != 0:
            for n in range(lower,upper+1):
                for tweet in tweets:
                    if self.rm_user_url:
                        tweet=tweet.replace("USER","")
                        tweet=tweet.replace("URL","")

                    ## word n-grams
                    for gram in nltk.ngrams(tweet.split(" "), n):
                        gram = "{}{}".format(PREFIX_WORD_NGRAM, " ".join(gram))
                        if self.binary:
                            d[gram] = 1 #binary
                        else:
                            d[gram] = d.get(gram,0)+1

        c_lower, c_upper = self.char_ngram_size
        if c_lower != 0:
            for n in range(c_lower, c_upper + 1):
                for tweet in tweets:
                    if self.rm_user_url:
                        tweet = tweet.replace("USER", "")
                        tweet = tweet.replace("URL", "")

                    ## char n-grams
                    for gram in nltk.ngrams(tweet, n):
                        gram = "{}_{}".format(PREFIX_CHAR_NGRAM, "_".join(gram))
                        if self.binary:
                            d[gram] = 1  # binary
                        else:
                            d[gram] = d.get(gram, 0) + 1

        return d

In [44]:
df = pd.read_csv('./data/surface_web.csv')

df.head()

df_surface = pd.read_csv("./data/surface_web_Forum.csv")

df_dark = pd.read_csv("./data/DarkWeb_Covid_Forum.csv")

df_surface['target'] = [0]*len(df_surface)
df_dark['target'] = [1]*len(df_dark)

list_df = [df_surface, df_dark]
df = pd.concat(list_df)

In [45]:
df.Main_Class.value_counts()

Forum    1260
Name: Main_Class, dtype: int64

In [46]:
#sentences = df.text.values

sentences = df.Text.values


#sentences = df.text.values

sentences = df.Text.values


target_clean_train = []

for x in df.target:
    if x == 'Forum' or x == 0:
        target_clean_train.append(0)
    if x == 'Market' or x == 1:
        target_clean_train.append(1)
        
labels = np.array(target_clean_train)


In [47]:
random_state = 1024

X_inputs, test_inputs, X_labels, test_labels = train_test_split(sentences, labels, random_state=random_state, test_size=0.55)

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(X_inputs, X_labels, random_state=random_state, test_size=0.01)



sentences_train = train_inputs
labels_train = train_labels


sentences_test = test_inputs
labels_test = test_labels


freqs = initFreq(sentences_test)

# print(len(labels_train))
# print(df_train.text[218][:20])
# print(bleachText(df_train.text[218][:20],"all", freqs))
print(bleachText("1x Pcs Mobile Case!? US$65","all", None))



02 03 06 06 05 W W W WPP W W W W W!? W$W dl ull ull ullxx uuxdd oc ccc cvcvcv cvcvoo vcooo 


In [48]:
def bleacher(input_data):
    temp = []
    for text in input_data:
        temp.append(bleachText(text,"all", freqs))
    return temp

bleached_train = bleacher(sentences_train)
# bleached_valid = bleacher(validation.text)
bleached_test = bleacher(sentences_test)
bleached_train[1]

'02 03 07 02 04 01 08 06 07 03 05 03 03 07 010 08 W W W W W W WP W W WP W W W W PW WP W W W W W W W, W W W. W W W W /W/W W. ul ll ll ll ll l llx ll ll llx ull ll ll ll xlxuudd llx vc cvv cvcvvcv cv cvcv v cvccvcvo ccvvcv cvccvcc vco ccvcc cvv cvc cvvcvcc ococvcvcoo cvccvvco '

In [49]:
n_gram_size = "1"

dictVectorizer = DictVectorizer()
vectorizerWords = Featurizer(word_ngrams=n_gram_size, binary= True)
X_train_dict = vectorizerWords.fit_transform(bleached_train)
X_test_dict = vectorizerWords.fit_transform(bleached_test)

X_train = dictVectorizer.fit_transform(X_train_dict)
X_test = dictVectorizer.transform(X_test_dict)
# print(X_train)

classifier = LinearSVC()
classifier.fit(X_train, labels_train)

Yguess = classifier.predict(X_test)
print(accuracy_score(labels_test, Yguess))



lowest = sorted(zip(classifier.coef_[0], dictVectorizer.get_feature_names()))[:100]
highest = sorted(zip(classifier.coef_[0], dictVectorizer.get_feature_names()), reverse=True)[:100]
feats = dict([(e[1], {}) for e in highest + lowest])
# print(feats)
# extract_feats("0", True, feats, sentences_train,sentences_train)


print(accuracy_score(labels_test, Yguess))
for i,j in highest[:15]:
#     print(f"{i:.2f}===> {j}")
    print(f"{j}")

0.9292929292929293
0.9292929292929293
 W
 
 ccvcccvc
 cvccvcvvc
 cvcccvcv
 ddxdd
 W-W
 vccvccc
 “W
 ccvcccvccvvc
 cvcvcvcvvcc
 –
 vccvcco
 vccc
 cvvccvc


In [50]:
n_gram_size = "5"

tfidfVectorizer = TfidfVectorizer(analyzer='word',ngram_range=(1,1), lowercase=False)
X_train_tf = tfidfVectorizer.fit_transform(bleached_train).toarray()
X_test_tf = tfidfVectorizer.transform(bleached_test).toarray()



In [51]:
classifier = LinearSVC()
classifier.fit(X_train_tf, labels_train)

Yguess = classifier.predict(X_test_tf)
print(accuracy_score(labels_test, Yguess))


lowest = sorted(zip(classifier.coef_[0], tfidfVectorizer.get_feature_names()))[:100]
highest = sorted(zip(classifier.coef_[0], tfidfVectorizer.get_feature_names()), reverse=True)[:100]
feats = dict([(e[1], {}) for e in highest + lowest])
# print(feats)
# extract_feats("0", True, feats, sentences_train, sentences_train)


print(accuracy_score(labels_test, Yguess))
for i,j in highest[:20]:
#     print(f"{i:.2f}===> {j}")
    print(f"{j}")


0.9105339105339105
0.9105339105339105
cvccvcvcvvc
vccvcc
cvcvccvc
cvcvcvvcc
ullx
vccvcvcvco
cvcccvcv
cvcvcvcvvcc
cvvcccvvc
vccvcvcvcc
ccvcccvc
cvccvv
ccvvc
ccv
xllx
vcccvcvccv
cvcvc
cvccvcvvc
vcv
uux


In [52]:

def extract_feats(c_n_gram,rm_uu, feats,original,transformed,n_gram =n_gram_size):
    """
    get mapping between transformed and original features (to later extract most frequent ones)
    (assumes same strategy as in Featurizer)
    """
    lower, upper = get_size_tuple(n_gram)
    if c_n_gram != "0":
        c_lower, c_upper = get_size_tuple(c_n_gram)
    for otext,ttext in zip(original,transformed):
        for otweet,ttweet in zip(otext.split(TWEET_DELIMITER),ttext.split(TWEET_DELIMITER)):
            if len(otweet.split(' ')) != len(ttweet.split(' ')):
                print('DIFF LEN:', otweet, ttweet)
                continue
            # word n-grams
            for n in range(lower, upper + 1):
                if rm_uu:
                    otweet = otweet.replace("USER", "")
                    otweet = otweet.replace("URL", "")
                    ttweet = ttweet.replace("USER", "")
                    ttweet = ttweet.replace("URL", "")
                for ongram,tngram in zip(nltk.ngrams(otweet.split(" "), n),nltk.ngrams(ttweet.split(" "), n)):
                    #print(ongram, tngram)
                    tngram = "{}_{}".format(PREFIX_WORD_NGRAM, " ".join(tngram))
                    ongram=' '.join(ongram).lower()
                    if tngram in feats:
                        feats[tngram][ongram]=feats[tngram].get(ongram,0)+1
            if c_n_gram != "0":
                ## character n-grams
                for n in range(c_lower, c_upper + 1):
                    if rm_uu:
                        otweet = otweet.replace("USER", "")
                        otweet = otweet.replace("URL", "")
                        ttweet = ttweet.replace("USER", "")
                        ttweet = ttweet.replace("URL", "")
                    for ongram, tngram in zip(nltk.ngrams(otweet, n), nltk.ngrams(ttweet, n)):
                        # print(ongram, tngram)
                        tngram = "{}_{}".format(PREFIX_CHAR_NGRAM, "_".join(tngram))
                        ongram = ' '.join(ongram).lower()
                        if tngram in feats:
                            feats[tngram][ongram] = feats[tngram].get(ongram, 0) + 1


In [53]:

classifier = LinearSVC()
classifier.fit(X_train, labels_train)
Yguess = classifier.predict(X_test)


lowest = sorted(zip(classifier.coef_[0], dictVectorizer.get_feature_names()))[:100]
highest = sorted(zip(classifier.coef_[0], dictVectorizer.get_feature_names()), reverse=True)[:100]
feats = dict([(e[1], {}) for e in highest + lowest])
# print(feats)
extract_feats("0", True, feats, sentences_train, sentence_train)


output_dir= "aria.txt"
OUT = open(output_dir, "w")
for (coef1, pattern1), (coef2, pattern2) in zip(lowest, highest):
    OUT.write(str(coef1) + ' ' + pattern1)
    OUT.write((80 - len(str(coef1) + ' ' + pattern1)) * ' ')
    OUT.write(str(coef2) + ' ' + pattern2 + '\n')
#     print(coef1)
    for (a1, b1), (a2, b2) in zip(sorted(feats[pattern1].items(), key=lambda x: -x[1])[:100],
                                  sorted(feats[pattern2].items(), key=lambda x: -x[1])[:100]):
#         print(a1,a2)
        OUT.write(a1 + (80 - len(a1)) * ' ')
        OUT.write(a2 + '\n')
    OUT.write('\n')

print(accuracy_score(labels_test, Yguess))
for i,j in highest[:15]:
    print(f"{i:.2f}===>{j}")

NameError: name 'sentence_train' is not defined

In [54]:
# print(train.text)
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
# tf_idf = CountVectorizer()
tf_idf = TfidfVectorizer()
X_train_tf = tf_idf.fit_transform(sentences_train).toarray()
X_test_tf = tf_idf.transform(sentences_test).toarray()

# print(X)
# X_train_counts = count_vect.fit_transform(train)#.toarray()

# train = X_train_counts
classifier = LinearSVC()
classifier.fit(X_train_tf, labels_train)

Yguess = classifier.predict(X_test_tf)
print(accuracy_score(labels_test, Yguess))

0.9264069264069265


In [32]:
highest = sorted(zip(classifier.coef_[0], tf_idf.get_feature_names()), reverse=True)[:100]
for i,j in highest[:10]:
    print(f"{i:.2f}===>{j}")
    

1.44===>covid
1.40===>mask
1.29===>face
0.94===>quality
0.87===>protection
0.82===>anti
0.81===>kn95
0.80===>19
0.77===>99
0.75===>ear
